# Классификация текста

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/My\ Drive

/content/drive/My Drive


In [ ]:
cd Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('lenta-ru-train.csv')

In [ ]:
df.head()

,title,text,topic,topic_label
0,"Инфляция в январе 2006 года составит 2,6 процента","Глава Росстата Владимир Соколин заявил, что в ...",Экономика,0
1,Никита Михалков учредил День российского кино,У российских кинематографистов появится новый...,Культура,3
2,Марко Матерацци вернется в строй к матчу с ЦСКА,"Медицинский штаб миланского ""Интера"" обнародов...",Спорт,1
3,"Определены лауреаты премии ""Книга года""","Премии ""Книга года"" в 13 номинациях вручены на...",Культура,3
4,Гол Роналду со штрафного спас португальцев от ...,Сборная Португалии сыграла вничью с командой И...,Спорт,1


In [43]:
df.topic_label.isna().sum()

0

In [ ]:
df.shape

(206637, 4)

In [ ]:
df['text'] = df['text'].str.replace('/',' ')

In [ ]:
df.topic.value_counts(normalize = True)

Экономика          0.307646
Спорт              0.250096
Культура           0.208443
Наука и техника    0.205113
Бизнес             0.028703
Name: topic, dtype: float64

In [ ]:
df.topic.isna().sum()

0

In [ ]:
import re

In [ ]:

# функция удаления пунктуации и цифр
def remove_trash(list): 
    pattern = '[^А-Яа-яЁё ]+'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
    except Exception as e:
      print(e)
    return list

df['text_clean'] = remove_trash(df['text'])

expected string or bytes-like object


In [ ]:
df = df[~df.text_clean.isna()].copy()

In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 8.2MB 15.2MB/s 


In [ ]:
import pymorphy2

In [ ]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Используем TF-IDF для подготовки фич
stop_russian = stopwords.words('russian')
text_transformer = TfidfVectorizer(stop_words=stop_russian, ngram_range=(1, 1), lowercase=True, max_features=15000)
text = text_transformer.fit_transform(df['text_clean'])

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(text, df['topic_label'], test_size=0.25, random_state=777)

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=10)

In [ ]:
clf.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
pred = clf.predict(X_test)

In [44]:
import sklearn
sklearn.metrics.f1_score(y_test,pred,average='weighted')

0.6383784308992175

# Приведем слова к первой форме. Внимание! Считается долго.

In [ ]:
morph = pymorphy2.MorphAnalyzer()

# функция лемматизации
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=3:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)


df['text_clean_normal'] = df.apply(lemmatize, axis = 1)

In [ ]:
# Используем TF-IDF для подготовки фич
stop_russian = stopwords.words('russian')
text_transformer = TfidfVectorizer(stop_words=stop_russian, ngram_range=(1, 1), lowercase=True, max_features=15000)
text = text_transformer.fit_transform(df['text_clean_normal'])

X_train, X_test, y_train, y_test = train_test_split(text, df['topic_label'], test_size=0.25, random_state=777)

clf.fit(X_train,y_train)
pred = clf.predict(X_test)

sklearn.metrics.f1_score(y_test,pred,average='weighted')